# SHAP feature selection #
## Code to select feature with combination of LightGBM and SHAP ##

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [8]:
number_of_folds = 5 # this sets global setting of which how many bootstraps to use
repeats = 10
#first round of optimization
lgb_params = {
 "objective": "regression",
 'boosting_type': 'gbdt', 
 'metric': ['mae','mse', 'huber'],
 'lambda_l1': 0.03674546022666247, 
 'lambda_l2': 2.5025758383109715, 
 'max_leaves': 22,
 'max_depth': 6, 
 'feature_fraction': 0.741996402547356, 
 'bagging_fraction': 0.8929839112500518, 
 'learning_rate': 0.0892243160116563, 
 'min_data_in_leaf': 5, 
 'drop_rate': 0.14842616274718734    
}

debug_local = True #to use local version

In [9]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.models import Metrics
from yspecies.selection import ShapSelector

In [12]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [13]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [14]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12340)",12340,39,445,"(12340, 2)","(40, 19)"


## Setting up SHAP selection pipeline ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [15]:
partition_params = PartitionParameters(number_of_folds, 0, 2, [],  42)


In [16]:
partition_shap_pipe = Pipeline([
    ("partitioner", DataPartitioner()),
    ('prepare_for_partitioning', TupleWith(lgb_params)),
    ("shap_computation", ShapSelector())
]
)
repeated_cv =  Repeat(partition_shap_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))

In [17]:
selection_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("partition_shap", repeated_cv)]
    )

## Setting up features to select ##

In [18]:
selection = select_lifespan = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])

In [19]:
select_lifespan = selection
select_mass = replace(selection, to_predict = "mass_g")
select_gestation = replace(selection, to_predict = "gestation")
select_mtgc = replace(selection, to_predict = "mtgc")

# First stage selection #

In [20]:
stage_one_lifespan = selection_pipeline.fit_transform((data, select_lifespan))
stage_one_lifespan

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	valid_0's l1: 8.95509	valid_0's l2: 189.944	valid_0's huber: 7.66501
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[138]	valid_0's l1: 3.65986	valid_0's l2: 70.135	valid_0's huber: 2.98298
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[77]	valid_0's l1: 4.57755	valid_0's l2: 31.7151	valid_0's huber: 3.74263
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[99]	valid_0's l1: 5.60866	valid_0's l2: 129.762	valid_0's huber: 4.7072
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 5.86207	valid_0's l2: 103.254	valid_0's huber: 4.92927
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 5.86207	valid_0's l2: 103.254	valid_0's huber: 4.92927


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[186]	valid_0's l1: 4.53548	valid_0's l2: 78.8119	valid_0's huber: 3.78829
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[98]	valid_0's l1: 3.60277	valid_0's l2: 74.9152	valid_0's huber: 2.95564
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[85]	valid_0's l1: 3.01916	valid_0's l2: 31.7315	valid_0's huber: 2.44062
Training until validation scores don't improve for 10 rounds
[200]	valid_0's l1: 5.69179	valid_0's l2: 87.4646	valid_0's huber: 4.81568
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 5.69179	valid_0's l2: 87.4646	valid_0's huber: 4.81568
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[129]	valid_0's l1: 3.38114	valid_0's l2: 39.5009	valid_0's huber: 2.77319


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[166]	valid_0's l1: 21.5694	valid_0's l2: 2027.76	valid_0's huber: 19.086
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[124]	valid_0's l1: 5.60692	valid_0's l2: 83.5029	valid_0's huber: 4.70877
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[103]	valid_0's l1: 2.8651	valid_0's l2: 24.2581	valid_0's huber: 2.25714
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[96]	valid_0's l1: 3.01419	valid_0's l2: 23.3583	valid_0's huber: 2.37319
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[7]	valid_0's l1: 10.193	valid_0's l2: 164.261	valid_0's huber: 8.78566


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[36]	valid_0's l1: 8.23239	valid_0's l2: 214.728	valid_0's huber: 7.03237
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[86]	valid_0's l1: 1.77836	valid_0's l2: 13.3965	valid_0's huber: 1.33522
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	valid_0's l1: 11.2635	valid_0's l2: 315.795	valid_0's huber: 9.73304
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[180]	valid_0's l1: 6.21307	valid_0's l2: 129.088	valid_0's huber: 5.27803
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[80]	valid_0's l1: 2.74116	valid_0's l2: 19.5057	valid_0's huber: 2.15102


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[118]	valid_0's l1: 2.69672	valid_0's l2: 24.7225	valid_0's huber: 2.14402
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[128]	valid_0's l1: 1.73385	valid_0's l2: 13.7157	valid_0's huber: 1.29137
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[124]	valid_0's l1: 2.70245	valid_0's l2: 17.2727	valid_0's huber: 2.10128
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[136]	valid_0's l1: 3.11295	valid_0's l2: 31.1112	valid_0's huber: 2.50236
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[33]	valid_0's l1: 5.24486	valid_0's l2: 82.1457	valid_0's huber: 4.34352


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


[<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
 <style scoped>
     .dataframe tbody tr th:only-of-type {
         vertical-align: middle;
     }
 
     .dataframe tbody tr th {
         vertical-align: top;
     }
 
     .dataframe thead th {
         text-align: right;
     }
 </style>
 <table border="1" class="dataframe">
   <thead>
     <tr style="text-align: right;">
       <th></th>
       <th>symbol</th>
       <th>gain_score_to_lifespan</th>
       <th>kendall_tau_to_lifespan</th>
     </tr>
     <tr>
       <th>ensembl_id</th>
       <th></th>
       <th></th>
       <th></th>
     </tr>
   </thead>
   <tbody>
     <tr>
       <th>ENSG00000276293</th>
       <td>PIP4K2B</td>
       <td>512897.728</td>
       <td>0.040</td>
     </tr>
     <tr>
       <th>ENSG00000083896</th>
       <td>YTHDC1</td>
       <td>192584.667</td>
       <td>0.179</td>
     </tr>
     <tr>
       <th>ENSG00000167515</th>
       <td>TRAPPC2L</td>
       <td>132820.645</td>
       <td>-0.220</td>
     </tr>
     <tr>
       <th>ENSG00000060762</th>
       <td>MPC1</td>
       <td>111991.538</td>
       <td>-0.308</td>
     </tr>
     <tr>
       <th>ENSG00000105672</th>
       <td>ETV2</td>
       <td>39996.305</td>
       <td>0.695</td>
     </tr>
     <tr>
       <th>ENSG00000142541</th>
       <td>RPL13A</td>
       <td>9135.891</td>
       <td>0.007</td>
     </tr>
     <tr>
       <th>ENSG00000065268</th>
       <td>WDR18</td>
       <td>3898.028</td>
       <td>-0.543</td>
     </tr>
     <tr>
       <th>ENSG00000101190</th>
       <td>TCFL5</td>
       <td>3801.633</td>
       <td>0.635</td>
     </tr>
     <tr>
       <th>ENSG00000066923</th>
       <td>STAG3</td>
       <td>2066.474</td>
       <td>0.516</td>
     </tr>
     <tr>
       <th>ENSG00000164879</th>
       <td>CA3</td>
       <td>1187.993</td>
       <td>-0.519</td>
     </tr>
     <tr>
       <th>ENSG00000171121</th>
       <td>KCNMB3</td>
       <td>463.055</td>
       <td>0.632</td>
     </tr>
     <tr>
       <th>ENSG00000118363</th>
       <td>SPCS2</td>
       <td>456.235</td>
       <td>0.221</td>
     </tr>
     <tr>
       <th>ENSG00000152705</th>
       <td>CATSPER3</td>
       <td>455.561</td>
       <td>0.555</td>
     </tr>
     <tr>
       <th>ENSG00000100764</th>
       <td>PSMC1</td>
       <td>251.336</td>
       <td>-0.660</td>
     </tr>
   </tbody>
 </table>
 </div></th><th><div>
 <style scoped>
     .dataframe tbody tr th:only-of-type {
         vertical-align: middle;
     }
 
     .dataframe tbody tr th {
         vertical-align: top;
     }
 
     .dataframe thead th {
         text-align: right;
     }
 </style>
 <table border="1" class="dataframe">
   <thead>
     <tr style="text-align: right;">
       <th></th>
       <th>R^2</th>
       <th>MSE</th>
       <th>MAE</th>
       <th>validation_species</th>
     </tr>
   </thead>
   <tbody>
     <tr>
       <th>0</th>
       <td>0.549</td>
       <td>189.944</td>
       <td>8.955</td>
       <td>[Phascolarctos_cinereus, Mus_musculus]</td>
     </tr>
     <tr>
       <th>1</th>
       <td>0.912</td>
       <td>70.135</td>
       <td>3.660</td>
       <td>[Equus_caballus, Rhinolophus_ferrumequinum]</td>
     </tr>
     <tr>
       <th>2</th>
       <td>0.933</td>
       <td>31.715</td>
       <td>4.578</td>
       <td>[Rattus_norvegicus, Mesocricetus_auratus]</td>
     </tr>
     <tr>
       <th>3</th>
       <td>0.769</td>
       <td>129.762</td>
       <td>5.609</td>
       <td>[Meriones_unguiculatus, Tursiops_truncatus]</td>
     </tr>
     <tr>
       <th>4</th>
       <td>0.809</td>
       <td>103.254</td>
       <td>5.862</td>
       <td>[Tupaia_belangeri, Cavia_aperea]</td>
     </tr>
   </tbody>
 </table>
 </div></th></tr></table>,
 <table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
 <style

In [21]:
from yspecies.results import FeatureSummary
summary = FeatureSummary(stage_one_lifespan)
stage_one_lifespan[0]

<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>symbol</th>
      <th>gain_score_to_lifespan</th>
      <th>kendall_tau_to_lifespan</th>
    </tr>
    <tr>
      <th>ensembl_id</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ENSG00000276293</th>
      <td>PIP4K2B</td>
      <td>512897.728</td>
      <td>0.040</td>
    </tr>
    <tr>
      <th>ENSG00000083896</th>
      <td>YTHDC1</td>
      <td>192584.667</td>
      <td>0.179</td>
    </tr>
    <tr>
      <th>ENSG00000167515</th>
      <td>TRAPPC2L</td>
      <td>132820.645</td>
      <td>-0.220</td>
    </tr>
    <tr>
      <th>ENSG00000060762</th>
      <td>MPC1</td>
      <td>111991.538</td>
      <td>-0.308</td>
    </tr>
    <tr>
      <th>ENSG00000105672</th>
      <td>ETV2</td>
      <td>39996.305</td>
      <td>0.695</td>
    </tr>
    <tr>
      <th>ENSG00000142541</th>
      <td>RPL13A</td>
      <td>9135.891</td>
      <td>0.007</td>
    </tr>
    <tr>
      <th>ENSG00000065268</th>
      <td>WDR18</td>
      <td>3898.028</td>
      <td>-0.543</td>
    </tr>
    <tr>
      <th>ENSG00000101190</th>
      <td>TCFL5</td>
      <td>3801.633</td>
      <td>0.635</td>
    </tr>
    <tr>
      <th>ENSG00000066923</th>
      <td>STAG3</td>
      <td>2066.474</td>
      <td>0.516</td>
    </tr>
    <tr>
      <th>ENSG00000164879</th>
      <td>CA3</td>
      <td>1187.993</td>
      <td>-0.519</td>
    </tr>
    <tr>
      <th>ENSG00000171121</th>
      <td>KCNMB3</td>
      <td>463.055</td>
      <td>0.632</td>
    </tr>
    <tr>
      <th>ENSG00000118363</th>
      <td>SPCS2</td>
      <td>456.235</td>
      <td>0.221</td>
    </tr>
    <tr>
      <th>ENSG00000152705</th>
      <td>CATSPER3</td>
      <td>455.561</td>
      <td>0.555</td>
    </tr>
    <tr>
      <th>ENSG00000100764</th>
      <td>PSMC1</td>
      <td>251.336</td>
      <td>-0.660</td>
    </tr>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MSE</th>
      <th>MAE</th>
      <th>validation_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.549</td>
      <td>189.944</td>
      <td>8.955</td>
      <td>[Phascolarctos_cinereus, Mus_musculus]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.912</td>
      <td>70.135</td>
      <td>3.660</td>
      <td>[Equus_caballus, Rhinolophus_ferrumequinum]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.933</td>
      <td>31.715</td>
      <td>4.578</td>
      <td>[Rattus_norvegicus, Mesocricetus_auratus]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.769</td>
      <td>129.762</td>
      <td>5.609</td>
      <td>[Meriones_unguiculatus, Tursiops_truncatus]</td>
    </tr>
    <tr>
      <th>4</th>
      <td>0.809</td>
      <td>103.254</td>
      <td>5.862</td>
      <td>[Tupaia_belangeri, Cavia_aperea]</td>
    </tr>
  </tbody>
</table>
</div></th></tr></table>

In [22]:
summary.selected.sort_values("kendall_tau_0", ascending=False)["symbol"].values

AttributeError: 'DataFrame' object has no attribute 'selected'

In [73]:
"gain" if len([c for c in summary.results[0].selected.columns if "gain" in c])>0 else "shap"

'gain'